In [3]:
import pandas as pd
import numpy as np
#from mlxtend.preprocessing import minmax_scaling
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor

### Load data

In [4]:
train_df=pd.read_csv('train.csv')

In [5]:
X_train = train_df.drop(["ID", "target"], axis=1)
y_train = train_df["target"]  # take log to train target

In [6]:
X_train.head()

,48df886f9,0deb4b6a8,34b15f335,a8cb14b00,2f0771a37,30347e683,d08d1fbe3,6ee66e115,20aa07010,dc5a8f1d8,...,3ecc09859,9281abeea,8675bec0b,3a13ed79a,f677d4d13,71b203550,137efaa80,fb36b89d9,7e293fbaf,9fc776466
0,0.0,0,0.0,0,0,0,0,0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
1,0.0,0,0.0,0,0,0,0,0,2200000.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
2,0.0,0,0.0,0,0,0,0,0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
3,0.0,0,0.0,0,0,0,0,0,0.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0
4,0.0,0,0.0,0,0,0,0,0,2000000.0,0.0,...,0.0,0.0,0.0,0,0,0,0,0,0,0


In [7]:
colsToRemove = []
for col in X_train.columns:
    if X_train[col].std() == 0: 
        colsToRemove.append(col)
X_train.drop(colsToRemove, axis=1, inplace=True)

# remove constant columns in the test set
#X_test.drop(colsToRemove, axis=1, inplace=True) 

print("Removed `{}` Constant Columns\n".format(len(colsToRemove)))
print(colsToRemove)

print("Train set size: {}".format(X_train.shape))
# print("Test set size: {}".format(X_test.shape))

Removed `256` Constant Columns

['d5308d8bc', 'c330f1a67', 'eeac16933', '7df8788e8', '5b91580ee', '6f29fbbc7', '46dafc868', 'ae41a98b6', 'f416800e9', '6d07828ca', '7ac332a1d', '70ee7950a', '833b35a7c', '2f9969eab', '8b1372217', '68322788b', '2288ac1a6', 'dc7f76962', '467044c26', '39ebfbfd9', '9a5ff8c23', 'f6fac27c8', '664e2800e', 'ae28689a2', 'd87dcac58', '4065efbb6', 'f944d9d43', 'c2c4491d5', 'a4346e2e2', '1af366d4f', 'cfff5b7c8', 'da215e99e', '5acd26139', '9be9c6cef', '1210d0271', '21b0a54cb', 'da35e792b', '754c502dd', '0b346adbd', '0f196b049', 'b603ed95d', '2a50e001c', '1e81432e7', '10350ea43', '3c7c7e24c', '7585fce2a', '64d036163', 'f25d9935c', 'd98484125', '95c85e227', '9a5273600', '746cdb817', '6377a6293', '7d944fb0c', '87eb21c50', '5ea313a8c', '0987a65a1', '2fb7c2443', 'f5dde409b', '1ae50d4c3', '2b21cd7d8', '0db8a9272', '804d8b55b', '76f135fa6', '7d7182143', 'f88e61ae6', '378ed28e0', 'ca4ba131e', '1352ddae5', '2b601ad67', '6e42ff7c7', '22196a84c', '0e410eb3d', '992e6d1d3', '90a7

#### Normalize dataset so that every value is in the same range 

In [44]:
from sklearn.preprocessing import StandardScaler
X_train=StandardScaler().fit_transform(X_train.values)

In [8]:
# Split training dataset and test dataset
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, 
                                                    test_size = 0.2, random_state = 42)

/Users/pdhe/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [9]:

## Xgboost regressor
import xgboost as xgb
xlf = xgb.XGBRegressor(max_depth=10, 
                        learning_rate=0.1, 
                        n_estimators=10, 
                        silent=True, 
                        objective='reg:linear', 
                        nthread=-1, 
                        gamma=0,
                        min_child_weight=1, 
                        max_delta_step=0, 
                        subsample=0.85, 
                        colsample_bytree=0.7, 
                        colsample_bylevel=1, 
                        reg_alpha=0, 
                        reg_lambda=1, 
                        scale_pos_weight=1, 
                        seed=1440, 
                        missing=None)

xlf.fit(X_train, y_train, eval_metric='rmse', verbose = True, eval_set = [(X_test, y_test)],early_stopping_rounds=100)

# 计算 auc 分数、预测


[0]	validation_0-rmse:8.94374e+06
Will train until validation_0-rmse hasn't improved in 100 rounds.
[1]	validation_0-rmse:8.55215e+06
[2]	validation_0-rmse:8.24316e+06
[3]	validation_0-rmse:7.94893e+06
[4]	validation_0-rmse:7.70254e+06
[5]	validation_0-rmse:7.50576e+06
[6]	validation_0-rmse:7.33335e+06
[7]	validation_0-rmse:7.20213e+06
[8]	validation_0-rmse:7.09016e+06
[9]	validation_0-rmse:6.99392e+06


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.7, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=10, min_child_weight=1, missing=None, n_estimators=10,
       n_jobs=1, nthread=-1, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=1440,
       silent=True, subsample=0.85)

In [10]:
preds = xlf.predict(X_test)

In [11]:
preds

array([  3055092.75 ,   2623424.   ,   4427569.5  ,   3083113.5  ,
         3083113.5  ,   5600924.5  ,   5905122.5  ,   4354622.5  ,
         3083113.5  ,   2776347.75 ,   7853575.   ,   3083113.5  ,
         5962904.   ,   3091903.75 ,   2644754.   ,   2898670.75 ,
         2870650.   ,   3627464.   ,   5723733.5  ,   4377585.5  ,
         2892089.25 ,   3083113.5  ,   3083113.5  ,   2898670.75 ,
         3083113.5  ,   8488786.   ,   3786191.   ,   2777491.   ,
         2499008.   ,   2593048.5  ,   4122797.5  ,   8514862.   ,
         3083113.5  ,   3083113.5  ,   2859514.   ,   2777491.   ,
         2154689.   ,   6628306.   ,   2350074.5  ,   7370660.   ,
         4243743.   ,   4122797.5  ,   2623314.75 ,   3314191.25 ,
         4263634.5  ,   2644754.   ,   4580838.   ,   4923251.   ,
         1870289.75 ,   5052579.   ,   5065876.5  ,   2870650.   ,
         3432977.75 ,   3564403.5  ,   3083113.5  ,   2423862.25 ,
        13034576.   ,   2182443.   ,   1878352.25 ,   5969404.

In [17]:
from sklearn import metrics
print("R2 Score is {}".format(metrics.r2_score(y_test,preds)))

R2 Score is 0.1374472731506553


In [15]:
def RMSLE(preds,y_test):
    diff=np.log(preds+1)-np.log(y_test+1)
    square_diff=np.square(diff)
    avg=np.mean(square_diff)
    return('RMSLE',np.sqrt(avg))

In [16]:
RMSLE(preds,y_test)

('RMSLE', 1.7068602643683972)

### without EDA RESULTS

In [20]:
import pandas as pd
import numpy as np
dataset = pd.read_csv('train.csv')
testset = pd.read_csv('test.csv')

X = dataset.iloc[:, 2:].values
y = dataset.iloc[:, 1].values
X_Test = testset.iloc[:,1:].values
y = np.log(y)

##get rid of features without variance
from sklearn.feature_selection import VarianceThreshold
feature_selector = VarianceThreshold()
X = feature_selector.fit_transform(X)
X_Test = feature_selector.transform(X_Test)

### XGboost Regressor

In [21]:
from xgboost import XGBRegressor
regressor = XGBRegressor(n_estimators=300)
regressor.fit(X, y)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=300,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [22]:
results = regressor.predict(X_Test)
results = np.exp(results)

### How to submit

In [23]:
submission = pd.DataFrame()
submission['ID'] = testset['ID']
submission['target'] = results
submission.to_csv('submission.csv', index=False)